In [275]:
import pandas as pd
import random
import numpy as np
from bs4 import BeautifulSoup
import requests
import time
import re

In [276]:
url = 'https://www.ebay.com/b/Computer-Processors-CPUs/164/bn_661751?_pgn=1'
page = requests.get(url)
b = BeautifulSoup(page.content,'html.parser')


In [277]:
price = []
title = []
shipprice = []
brand = []
reviewnum = []

def extract(l):
    global price, address, sqft, acres, beds, baths
    for i in range(len(l)):
        pricE = allitems[i].find_all(class_ = 's-item__price')
        titlE = allitems[i].find_all(class_ = 's-item__title')
        shipP = allitems[i].find_all(class_ = 's-item__shipping')
        branD = allitems[i].find_all(class_ = 's-item__dynamic')
        reviewS = allitems[i].find_all(class_ = 's-item__reviews-count')
        if len(pricE) == 0:
            price.append(0.00)
        else:
            price.append(float(re.findall('\d+',pricE[0].text)[0]))
            #price.append(pricE[0].text)
        if len(titlE) == 0:
            title.append('N/A')
        else:
            title.append(titlE[0].text)
        if (len(shipP) == 0) or ("Free" in shipP[0].text):
            shipprice.append(0.00)
        else:
            shipprice.append(float(shipP[0].text[1:-9]))
        if len(branD) == 0:
            brand.append("N/A")
        else:
            if "Brand:" not in branD[0].text:
                brand.append("N/A")
            else:
                brand.append(branD[0].text[7:])
        if (len(reviewS) == 0) or (reviewS[0].text == ''):
            reviewnum.append(0)
        elif reviewS[0].text == '1 product rating':
            reviewnum.append(1)
        else:
            reviewnum.append(int(re.findall('\d+',reviewS[0].text)[0]))
        
        
min_wait = 1
max_wait = 3
i = 1
print("Started.")
while len(price)<1200:
    url = 'https://www.ebay.com/b/Computer-Processors-CPUs/164/bn_661751?_pgn=%d'%(i)
    page = requests.get(url)
    b = BeautifulSoup(page.content,'html.parser')
    allitems = b.find_all(class_ = 's-item')
    time.sleep(random.randint(min_wait,max_wait))
    extract(allitems)
    i+=1
print("Finished.")

di = {'Brand':brand,'Description':title,'Price':price,'Shipping':shipprice,'Num_of_Reviews':reviewnum}
df = pd.DataFrame(di)
df['Total_Price'] = df.Price+df.Shipping
df['Popular'] = df.Num_of_Reviews >= 100

Started.
Finished.


In [280]:
df = df.sort_values(by = 'Num_of_Reviews',ascending = False)

print('Average price: $'+str(df.Price.mean()))
print('Average price of AMD processors: $'+str(df[df.Brand == 'AMD'].Price.mean()))
print('Average price of Intel processors: $'+str(df[df.Brand == 'Intel'].Price.mean()))
print('Standard deviation of the price: '+str(df.Price.std()))
print()
print('Number of processors collected: '+str(len(df.Brand)))
print('Number of Intel processors: '+str(len(df[df.Brand =='Intel'].Brand)))
print('\tAverage price of Intel i7 processors: $'+str(df[(df.Description.str.contains('i7')) & (df.Brand == 'Intel')].Price.mean()))
print('\t\tNumber of Intel i7 processors: '+str(len(df[(df.Description.str.contains('i7')) & (df.Brand == 'Intel')].Price)))
print('Number of AMD processors: '+str(len(df[df.Brand =='AMD'].Brand)))
df

Average price: $73.2475
Average price of AMD processors: $48.9047619048
Average price of Intel processors: $71.9810554804
Standard deviation of the price: 125.994994051

Number of processors collected: 1200
Number of Intel processors: 739
	Average price of Intel i7 processors: $197.172043011
		Number of Intel i7 processors: 93
Number of AMD processors: 231


,Brand,Description,Num_of_Reviews,Price,Shipping,Total_Price,Popular
330,Intel,New ListingIntel Core 2 Quad Q6600 2.4 GHz 8M ...,565,9.0,4.95,13.95,True
360,Intel,INTEL Q6600 Core 2 Quad Socket 775 CPU Processor,565,10.0,0.00,10.00,True
565,Intel,Genuine OEM Intel Core 2 Quad SLACR CPU 2.40GH...,565,9.0,0.00,9.00,True
1041,Intel,Intel Core 2 Quad Q6600 2.4 GHz 8M 1066 Quad-C...,565,13.0,0.00,13.00,True
131,Intel,New ListingUsed Intel Core i7-4790K 4GHz Quad-...,211,179.0,9.85,188.85,True
221,Intel,New ListingIntel Core i7-4790K 4GHz Quad-Core ...,211,261.0,11.10,272.10,True
161,Intel,New ListingIntel Core i7-4790K 4GHz Quad-Core ...,211,279.0,4.99,283.99,True
36,AMD,AMD FD8350FRHKBOX FX-8350 FX-Series 8-Core Bla...,192,126.0,0.00,126.00,True
78,AMD,AMD FX 8350 Black Edition 4GHz Eight Core (FD8...,192,125.0,0.00,125.00,True
1051,AMD,New ListingAMD FX 8350 Black Edition 4GHz Eigh...,192,118.0,0.00,118.00,True


In [279]:
df.to_csv('eBayProcessors.csv')